# Языковое моделирование
Курс NLP, лабораторная 1.  
Осень 2015.

В данном задании вы реализуете:

- Add-one smoothing
- Stupid backoff
- Interpolation smoothing
- EM-algorithm
- Kneser-Ney smoothing

Вы примените это к:

- Language recognition problem

**Подпись**: Тихонова Мария Ивановна - 2 курс - КН

Цель языкового моделирования заключается в том, чтобы присвоить некоторые вероятности предложениям. Возникает закономерный вопрос, а зачем нам это надо? Например, в задачах _машинного перевода_ частенько нужно среди нескольких предложений выбирать наиболее вероятный перевод (который является естественным для человеческого глаза). Также это чрезвычайно полезно в задаче _исправления опечаток_ и _распозновании речи_.

Наша задача состоит в подсчете вероятности $P(W) = P(w_1, \dots, w_n)$ или $P(w_n \mid w_1, \dots, w_{n-1})$. Модель, умеющая вычислять хотя бы одну из этих двух вероятностей, называется **язковой моделью** (LM от Language Model).

Пришло время вспомнить _цепное правило_ (chain rule) $P(X_1, \dots, X_n) = P(X_1)P(X_2 \mid X_1)\dots P(X_n \mid X_1, \dots, X_{n-1})$. Также мы знаем, что

$$
    P(X_n \mid X_1, \dots, X_{n-1}) = \frac{P(X_1, \dots, X_n)}{P(X_1, \dots, X_{n-1})},
$$

следовательно, для того чтобы оценить $P(X_n \mid X_1, \dots, X_{n-1})$ нужно посчитать $P(X_1, \dots, X_n)$ и $P(X_1, \dots, X_{n-1})$. Но эти вероятности будут чрезвычайно малы, если мы возьмем большое $n$, так множество предложений из $n$ слов растет экспоненциально. Для упрощения ситуации применим **марковское предположение**: $P(X_n \mid X_1, \dots, X_{n-1}) = P(X_n \mid X_{n - k + 1}, \dots, X_{n-1})$ для некоторого фиксированного (небольшого) $k$. Это предположение интуитивно ясно и говорит нам о том, что $X_{n}$ не зависит от $X_{1}, \dots, X_{n - k}$, то есть на следующее слово влияет лишь контекст из предыдущих $k - 1$ слова. Таким образом, мы получаем финальную вероятность:

$$
    P(w_1, \dots, w_n) = \prod_i P(w_i \mid w_{i-k+1}, \dots, w_{i - 1}).
$$

Далее для краткости будем обозначать $w_{i-k}^i := w_{i-k}, \dots, w_{i}$.

## Хранилище n-грам

Пришло время написать класс для хранения n-грамм. Следуйте комментариям, чтобы написать NGramStorage с удобным интерфейсом.

In [84]:
import re
import math
import random
from collections import Counter
import numpy as np
from sklearn.cross_validation import train_test_split

In [85]:
from nltk.util import ngrams

In [86]:
EPS = 10 ** (-50)

In [87]:
class NGramStorage:
    """Storage for ngrams' frequencies.
    
    Args:
        sents (list[list[str]]): List of sentences from which ngram
            frequencies are extracted.
        max_n (int): Upper bound of the length of ngrams.
            For instance if max_n = 2, then storage will store
            0, 1, 2-grams.
            
    Attributes:
        max_n (Readonly(int)): Upper bound of the length of ngrams.
    """
        
    def __init__(self, sents=[], max_n=0):
        self.__max_n = max_n
        self.__ngrams = {i: Counter() for i in range(self.__max_n + 1)}
        
        # self._ngrams[K] should have the following interface:
        # self._ngrams[K][(w_1, ..., w_K)] = number of times w_1, ..., w_K occured in words
        # self._ngrams[0][()] = number of all words
        
        ### YOUR CODE HERE
        
        for i in range(1, self.__max_n + 1):
            for sent in sents:
                for ngram in ngrams(sent, i):
                    self.__ngrams[i][ngram] += 1
        self.__ngrams[0][()] = sum(self.__ngrams[1].values())
            
            
            

        ### END YOUR CODE
        
    def add_unk_token(self):
        """Add UNK token to 1-grams."""
        # In order to avoid zero probabilites 
        if self.__max_n == 0 or u'UNK' in self.__ngrams[1]:
            return
        self.__ngrams[0][()] += 1
        self.__ngrams[1][(u'UNK',)] = 1
        
    @property
    def max_n(self):
        """Get max_n"""
        return self.__max_n
        
    def __getitem__(self, k):
        """Get dictionary of k-gram frequencies.
        
        Args:
            k (int): length of returning ngrams' frequencies.
            
        Returns:
            Dictionary (in fact Counter) of k-gram frequencies.
        """
        # Cheking the input
        if not isinstance(k, int):
            raise TypeError('k (length of ngrams) must be an integer!')
        if k > self.__max_n:
            raise ValueError('k (length of ngrams) must be less or equal to the maximal length!')
        return self.__ngrams[k]
    
    def __call__(self, ngram):
        """Return frequency of a given ngram.
        
        Args:
            ngram (tuple): ngram for which frequency should be computed.
            
        Returns:
            Frequency (int) of a given ngram.
        """
        # Cheking the input
        if not isinstance(ngram, tuple):
            raise TypeError('ngram must be a tuple!')
        if len(ngram) > self.__max_n:
            raise ValueError('length of ngram must be less or equal to the maximal length!')
        if len(ngram) == 1 and ngram not in self.__ngrams[1]:
            return self.__ngrams[1][(u'UNK', )]
        return self.__ngrams[len(ngram)][ngram]

Давайте скачаем корпус и запустим на нем наши эксперименты.

In [88]:
import nltk
# Uncomment next row and download brown corpus
# nltk.download()
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Maria\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [89]:
all_sents = list(brown.sents())
random.seed(123)
random.shuffle(all_sents)
print('Number of all sentences = {}'.format(len(all_sents)))
train_sents = all_sents[:int(0.8 * len(all_sents))]
test_sents = all_sents[int(0.8 * len(all_sents)):]
print('Number of train sentences = {}'.format(len(train_sents)))
print('Number of test sentences = {}'.format(len(test_sents)))

Number of all sentences = 57340
Number of train sentences = 45872
Number of test sentences = 11468


In [90]:
# Create storage of 0, 1, 2, 3-grams
storage = NGramStorage(train_sents, 3)

In [91]:
# It's time to test your code
print(storage(('to', 'be')))
print(storage(('or',)))
print(storage(('not', 'to', 'be')))
print(storage(('Muammar',)))
print(storage(()))

1371
3265
28
0
930601


## Оценка качества

Определим **перплексию**:

$$
    {\rm PP}(w_1, \dots, w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_i P(w_i \mid w_{i - k}, \dots, w_{i - 1})\right)^{-\frac1N},
$$

кстати, из этой формулы видно, что задача по минимизации перплексии равносильна задаче по максимизации вероятности.

Реализуйте функцию по подсчету перплексии. Стоит отметить, что перплексия по корпусу $-$ это не то же самое, что и усредненная перплексия по предложениям. Перплексия по корпусу равна произведению вероятностей **всех** предложений в степени $-\frac1N$, где $N -$ суммарная длина всех предложений.

In [92]:
def perplexity(estimator, sents):
    '''Estimate perplexity of the sequence of words using prob_estimator.'''
    ### YOUR CODE HERE
    # Avoid log(0) by replacing zero by 10 ** (-50).
    perp = 0.
    L = 0.
    for sent in sents:
        L += len(sent)
        prob = estimator.prob(sent)
        perp += math.log(prob + 10 ** (-50) * float(prob < 10 ** (-50)))
    # deg = sum(len(sent) for sent in sents)
    perp = math.exp(-1. / L * perp)
    ### END YOUR CODE
    
    return perp

## Оценка вероятностей n-грам

Первый и простейший способ оценки вероятностей N-грам следующий:

$$
    \hat P_{S}(w_{N} \mid w_1^{N - 1}) = \frac{c(w_1^N)}{c(w_1^{N-1})}.
$$

где $c(w_1^N)$ — это число последовательностей $w_1, \dots, w_N$ в корпусе, $S$ символизирует Straightforward. Что-ж, пора это реализовать.

In [93]:
class StraightforwardProbabilityEstimator:
    """Class for simplest probability estimations of type P(word | context).
    
    P(word | context) = c(context + word) / c(context), where
    c(sequence) - number of occurances of the sequence in the corpus.
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
    """
    
    def __init__(self, storage):
        self.__storage = storage
        # Adding UNK token to avoid zero probabilities
        self.__storage.add_unk_token()
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if self.__storage.max_n == 1:
            return ()
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not isinstance(word, str):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('word must be a string!')
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        phrase_counts = self.__storage(context + (word, ))
        context_counts = self.__storage(context)
        # Avoiding 0 / 0.
        if context_counts == 0:
            return 0.
        return 1. * phrase_counts / context_counts
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [94]:
# Initialize estimator
simple_estimator = StraightforwardProbabilityEstimator(storage)

# Estimating perplexity
print('Simple estimator perplexity = {}'.format(perplexity(simple_estimator, test_sents)))
print(simple_estimator.prob('To be'.split()))
print(simple_estimator.prob('To be or not to be'.split()))

Simple estimator perplexity = 265.54563079779234
1.9342318198327532e-05
0.0


Посчитаем перплексию униграмной модели.

In [95]:
uni_storage = NGramStorage(train_sents, 1)
uni_simple_estimator = StraightforwardProbabilityEstimator(uni_storage)
print('Simple estimator perplexity = {}'.format(perplexity(uni_simple_estimator, test_sents)))

Simple estimator perplexity = 108.52877490449828


___
<font color='red'>Ответьте на следующие вопросы (внутри ipython ноутбука):</font>

**Q:** Какие выводы можно сделать? Почему $P(\text{To be or not to be}) = 0$, хотя мы и добавили UNK токен?  
**A:**

**Q:** Почему перплексия униграмной модели меньше, чем триграмной?  
**A:**
___

Чтобы ответить на вопросы посмотрим, какие верятности у всех ngram из нашего предложения 'To be or not to be'.

In [96]:
sent = 'To be or not to be'
contex = tuple('To be or not to be'.split())
for i in range(len(contex)):
    print('Word {}'.format(contex[i]))
    print('Context {}'.format(contex[:i]))
    print('Probability for trigrams prediction {}'.format(simple_estimator(contex[i], contex[:i])))
    print('Probability for unigrams prediction {}'.format(uni_simple_estimator(contex[i], contex[:i])))
    print('\n')

print('Total probability for trigram model {}'.format(simple_estimator.prob(sent)))
print('Total probability for unigram model {}'.format(uni_simple_estimator.prob(sent)))

Word To
Context ()
Probability for trigrams prediction 0.0003707277654679444
Probability for unigrams prediction 0.0003707277654679444


Word be
Context ('To',)
Probability for trigrams prediction 0.05217391304347826
Probability for unigrams prediction 0.005476025196593173


Word or
Context ('To', 'be')
Probability for trigrams prediction 0.0
Probability for unigrams prediction 0.003508481606529967


Word not
Context ('To', 'be', 'or')
Probability for trigrams prediction 0.0
Probability for unigrams prediction 0.003790019793638956


Word to
Context ('To', 'be', 'or', 'not')
Probability for trigrams prediction 0.041666666666666664
Probability for unigrams prediction 0.02212976116535318


Word be
Context ('To', 'be', 'or', 'not', 'to')
Probability for trigrams prediction 0.20437956204379562
Probability for unigrams prediction 0.005476025196593173


Total probability for trigram model 0.0
Total probability for unigram model 5.255461077476345e-103


Теперь все понятно. Из-за того, что вероятность некоторых слов в данном контексте в исходном предложении равна 0, то и вероятность всего предложения, которая считается как произведение вероятностей, тоже равна нулю. Вывод:  у StraightforwardProbabilityEstimator есть один большой недостаток: если вероятность хотя бы какого-то слова в данном контексте равна нулю, то и вероятность всего предложения окажется нулевой, что не обязательно верно.

У униграммной модели этого недостатка нет, поэтому нулевых вероятностей мы не получаем (как видно из примера, вероятность модельного предложения для униграмной модели ненулевая), поэтому и перплексия меньше.

## Add-one smoothing

Простейший вид сглаживания — **сглаживание Лапласа**. Каким-то образом мы хотим избавиться от нулевых вероятностей. Наиболее простой алгоритм борьбы с этой проблемой следующий: для оценки $P(w_{N} \mid w_1^{N - 1})$ мы будем использовать формулу:

$$
    \hat P_{AOS}(w_{N} \mid w_1^{N - 1}) = \frac{c(w_1^N) + \delta}{c(w_1^{N-1}) + \delta V},
$$

где $V$ — это размер словаря, а $\delta$ — некоторая фиксированная константа.

Теперь пришло время снова пописать код. Реализуйте класс, симулирующий сглаживание Лапласа. Он должен иметь аналогичный интерфейс, как и StraightforwardProbabilityEstimator.

In [97]:
class LaplaceProbabilityEstimator:
    """Class for probability estimations of type P(word | context).
    
    P(word | context) = (c(context + word) + delta) / (c(context) + delta * V), where
    c(sequence) - number of occurances of the sequence in the corpus,
    delta - some constant,
    V - number of different words in corpus.
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
        delta(float): Smoothing parameter.
    """
    
    def __init__(self, storage, delta=1.):
        self.__storage = storage
        self.__delta = delta
        # Adding UNK token to avoid zero probabilities
        # self.__storage.add_unk_token()
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not isinstance(word, str):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('context must be a tuple!')
            
        ### YOUR CODE HERE
        context = self.cut_context(context)
        phrase_counts = self.__storage(context + (word, ))
        context_counts = self.__storage(context)
        # Avoiding 0 / 0.
        prob = 1.
        # if context_counts == 0: Эта строчка здесь не нужна, так как идея сноаживания как раз в том
        #чтобы уйти от нулевых вероятностей
            # return 0.
        return 1. * (phrase_counts + self.__delta)/ (context_counts +  self.__delta * len(self.__storage[1]))

        ### END YOUR CODE
        
        # return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

Подберите наилучший параметр $\delta$ для данного корпуса.

In [98]:
from scipy.optimize import minimize_scalar

In [99]:
def laplas_perplexity(x):
    return perplexity(LaplaceProbabilityEstimator(storage, abs(x)), test_sents)

In [100]:
best_delta = 1.
best_delta = minimize_scalar( lambda x: laplas_perplexity(x), bounds=(EPS, 1.0), method='bounded', ).x

print('Best delta equals = {}'.format(best_delta))
laplace_estimator = LaplaceProbabilityEstimator(storage, best_delta)

print('Laplace estimator perplexity = {}'.format(perplexity(laplace_estimator, test_sents)))
print(laplace_estimator.prob('To be'.split()))

Best delta equals = 0.0003861116037380595
Laplace estimator perplexity = 137.28000600290179
1.83054888539e-05


## Stupid backoff

Идея **тупого откатывания** невероятно проста (на то оно и тупое!). Если у нас есть достаточно информцаии для подсчета вероятности $k$-грам, тогда будем использовать $k$-грамы. Если эта вероятность крайне мала, то будем использовать вероятности $(k-1)$-грам с некоторым множителем, например, $0.4$, и так далее. К сожалению, в данном случае мы получим не вероятностное распределение, но в большинстве задач это не принципиально важно.

Реализуйте класс, симулирующий сглаживание тупым откатыванием. Он должен иметь аналогичный интерфейс, как и StraightforwardProbabilityEstimator.

In [101]:
class StupidBackoffProbabilityEstimator:
    """Class for stupid backoff probability estimations.
    
    P(word | context) =
        P'(word | context),                  if  P'(word | context) > 0;
        P'(word | context[1:]) * multiplier, if  P'(word | context) == 0
                                             and P'(word | context[1:]) > 0;
        ...
    P'(word | context) - probability of a word provided context of a base estimator.
    
    Args:
        base_estimator(BaseProbabilityEstimator): Object of BaseProbabilityEstimator
            or some other class which can estimate conditional probabilities.
        multiplier (float): Multiplier which is used for probability estimations.
    """
    
    def __init__(self, base_estimator, multiplier=0.1):
        self.__base_estimator = base_estimator
        self.__mult = multiplier
        
    def __backoff__(self, word, context):
        original_prob = self.__base_estimator(word, context)
        if original_prob > EPS:
            return original_prob
        else:
            return self.__mult * self.__backoff__(word, context[1:])
    
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        
        ### YOUR CODE HERE
        prob = self.__backoff__(word, context)
        ### END YOUR CODE
        
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [102]:
# Initialize estimator
sbackoff_estimator = StupidBackoffProbabilityEstimator(simple_estimator, .4)

# Let's make some estimations
print('Stupid backoff estimator perplexity = {}'.format(perplexity(sbackoff_estimator, test_sents)))
print(sbackoff_estimator.prob('To be'.split()))

Stupid backoff estimator perplexity = 123.53797910506032
1.9342318198327532e-05


___
<font color='red'>Ответьте на следующие вопросы (внутри ipython ноутбука):</font>

**Q:** Почему бессмысленно измерять перплексию в случае **Stupid backoff**?  
**A:**
___

Перплексию в данном случае считать бессмысленно, так как в случае Stupid backoff полученное распределение не вероятностное.

## Interpolation smoothing

В данном случае идея сглаживания посредством **интерполяции** также крайне проста. Пусть у нас есть $N$-грамная модель. Заведем вектор $\bar\lambda = (\lambda_1, \dots, \lambda_N)$, такой, что $\sum_i\lambda_i = 1$ и $\lambda_i \geq 0$. Ну а далее просто постулируем, что

$$
    \hat P_{IS}(w_{N} \mid w_1^{N-1}) = \sum_{i=1}^N \lambda_i \hat P_{S}(w_N \mid w_{N-i+1}^{N-1}).
$$

Придумайте, как обойтись одним вектором $\bar\lambda$. Казалось бы, их нужно несколько, ибо наша модель должна уметь считать вероятности $P(w_3 \mid w_1, w_2)$, а иногда $P(w_2 \mid w_1)$. Если мы тупо обрубим сумму, то у нас уже не будет вероятностное распределение, что, конечно же, плохо.

In [103]:
class InterpolationProbabilityEstimator:
    """Class for interpolation probability estimations.
    
    P(word | context) =
        lambda_N * P'(word | context) +
        lambda_{N-1} * P'(word | context[1:]) +
        ... +
        lambda_1 * P'(word)
    P'(word | context) - probability of a word provided context of a base estimator.
    
    Args:
        base_estimator(BaseProbabilityEstimator): Object of BaseProbabilityEstimator
            or some other class which can estimate conditional probabilities.
        lambdas (np.array[float]): Lambdas which are used for probability estimations.
    """
    
    def __init__(self, base_estimator, lambdas):
        self.lambdas = lambdas
        self.__base_estimator = base_estimator
        
    def __normilize_prob__(self, iterations, prob):
        sum = 0.0
        for i in range(iterations):
            sum += self.lambdas[i]
        return prob/sum
            
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        
        ### YOUR CODE HERE
        prob = 0.
        iterations = min(len(context)+1, len(self.lambdas))
        for i in range(iterations):
            prob += self.lambdas[i] * self.__base_estimator(word, context[len(context)-i:])
        #print(prob)
        prob = self.__normilize_prob__(iterations, prob)
        #print(prob)
        ### END YOUR CODE
        
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [104]:
import numpy as np

In [105]:
# Initialize estimator
interpol_estimator = InterpolationProbabilityEstimator(simple_estimator, np.array([0.2, 0.2, 0.6]))

# Let's make some estimations
print('Interpolation estimator perplexity = {}'.format(perplexity(interpol_estimator, test_sents)))
#print(laplace_estimator.prob('To be'.split()))
print('Probability of TO BE:')
print(interpol_estimator.prob('To be'.split()))

Interpolation estimator perplexity = 92.38161856595359
Probability of TO BE:
1.06862163916e-05


Остается один вопрос: как подбирать $\bar\lambda$? Для этого обычно применяется EM-алгоритм. Сразу опишем лишь готовую формулу. Пусть $\bar\lambda^t$ соответствует набору коэффициентов на $t$-м шаге. $\bar\lambda^0$ задаем произвольно.

* **E-step**:
$$
    \hat\lambda_j^t = \sum_{w_1^N} \frac{\lambda_j^t \cdot \hat P_{S}(w_N \mid w_{N-j+1}^{N-1})}{\sum_{i=1}^N \lambda_i^t \hat P_{S}(w_N \mid w_{N-i+1}^{N-1})}.
$$
* **M-step**:
$$
    \lambda_j^{t+1} = \frac{\hat\lambda_j^t}{\sum_{i=1}^N \hat\lambda_i^t}.
$$

Формулы выписаны, то бишь самое сложное сделано. Вам остается это лишь реализовать.

In [106]:
from numpy import linalg as LA

In [107]:
def E_step(test_storage, s_estimator, i_estimator):
    ### YOUR CODE HERE
    estimated_lambdas = np.zeros(test_storage.max_n)
    
    for ngam in test_storage[test_storage.max_n].keys():
        znam = sum(i_estimator.lambdas[i] * s_estimator(str(ngam[-1]), ngam[-i - 1:-1]) for i in range(test_storage.max_n))
        for j in range(test_storage.max_n):
            chisl = i_estimator.lambdas[j] * s_estimator(str(ngam[-1]), ngam[-j - 1:-1])
            estimated_lambdas[j] += chisl / znam
    ### END YOUR CODE
    
    return estimated_lambdas

def M_step(estimated_lambdas):
    new_lambdas = estimated_lambdas
    sum = np.sum(estimated_lambdas)
    for j in range(len(new_lambdas)):
        new_lambdas[j] /= sum
    return new_lambdas

def EM_algorithm(test_storage, s_estimator, i_estimator, epsilon=0.03):
    ### YOUR CODE HERE
    while True:
        old_lambdas = i_estimator.lambdas[:]
        estimated_lambdas = E_step(test_storage, s_estimator, i_estimator)
        i_estimator.lambdas = M_step(estimated_lambdas)
        norm = LA.norm(i_estimator.lambdas - old_lambdas)
        print(norm)
        if norm < epsilon:
            break
    ### END YOUR CODE
    
    return i_estimator.lambdas

In [108]:
# Separate train into train and test
train_set = train_sents[:int(0.6 * len(train_sents))]
test_set = train_sents[int(0.6 * len(train_sents)):]
MAX_N = 3
train_storage = NGramStorage(train_set, MAX_N)
test_storage = NGramStorage(test_set, MAX_N)
s_estimator = StraightforwardProbabilityEstimator(train_storage)

# Make starting assumption
starting_lambdas = np.array([0.33, 0.33, 0.34])
i_estimator = InterpolationProbabilityEstimator(s_estimator, starting_lambdas)

In [109]:
# It can take some time
best_lambdas = EM_algorithm(test_storage, s_estimator, i_estimator)

0.324386547703
0.0489980424294
0.01455548097


In [110]:
# Initialize estimator
i_estimator = InterpolationProbabilityEstimator(simple_estimator, best_lambdas)

# Let's make some estimations
print('Interpolation estimator perplexity = {}'.format(perplexity(i_estimator, test_sents)))
print(i_estimator.prob('To be'.split()))
print('Best lambdas = {}'.format(best_lambdas))

Interpolation estimator perplexity = 87.33784337510922
9.07135205045e-06
Best lambdas = [ 0.56881651  0.38995087  0.04123262]


## Kneser-Ney smoothing

Идея данного сглаживания заключается в том, что словам, которые участвуют в большом количестве контекстов, присваиваются большие вероятности, а те, которые исползуются в паре-тройке контекстов получают маленькие вероятности. Авторы данного сглаживания формализовали это следующим образом. Введем обозначения

$$
    N_{1+}(\cdot w_2) := \left|\{w_1 : c(w_1, w_2) > 0\}\right|,
$$
$$
    N_{1+}(\cdot \cdot) := \sum_{w_2} N_{1+}(\cdot w_2),
$$
$$
    \hat P_{KN} (w_1) = \frac{{\rm max}\{N_{1+}(\cdot w_1)-\delta,0\}}{N_{1+}(\cdot \cdot)} + \frac{\delta}{|V|}.
$$

Далее мы используем реккурентное соотношение

$$
    \hat P_{KN}(w_{N} \mid w_1^{N-1}) = \frac{{\rm max}\{c(w_1^N) - \delta, 0\}}{\sum_{w_N}c(w_1^{N-1}w_N)} + \frac{\delta}{\sum_{w_N}c(w_1^{N-1}w_N)}N_{1+}(w_1^{N-1}\cdot)\hat P_{KN}(w_N \mid w_2^{N-1}).
$$

Для вас дело за малым — реализовать это.

# При написании кода я пользовалась следующей, исправленной формулой:

 Идея данного сглаживания заключается в том, что словам, которые участвуют в большом количестве контекстов, присваиваются большие вероятности, а те, которые исползуются в паре-тройке контекстов получают маленькие вероятности. Авторы данного сглаживания формализовали это следующим образом. Введем обозначения

$$
 N_{1+}(\cdot w_2) := \left|\{w_1 : c(w_1, w_2) > 0\}\right|,
$$
$$
 N_{1+}(\cdot \cdot) := \sum_{w_2} N_{1+}(\cdot w_2),
$$
На семинаре я показывал, что надо изменить формулу:
$$
 \hat P_{KN} (w_1) = \frac{\max(N_{1+}(\cdot w_1) - \delta, 0)}{N_{1+}(\cdot \cdot)} + \frac{\delta \|\left\{ w \colon N_{1+}(\cdot w) > 0\right\}\|}{V N_{1+}(\cdot \cdot)}.
$$

Далее мы используем реккурентное соотношение

$$
 \hat P_{KN}(w_{N} \mid w_1^{N-1}) = \frac{{\rm max}\{c(w_1^N) - \delta, 0\}}{\sum_{w_N}c(w_1^{N-1}w_N)} + \frac{\delta}{\sum_{w_N}c(w_1^{N-1}w_N)}N_{1+}(w_1^{N-1}\cdot)\hat P_{KN}(w_N \mid w_2^{N-1}).
$$ 

In [111]:
from nltk import bigrams

In [112]:
class KneserNeyProbabilityEstimator:
    """Class for probability estimations of type P(word | context).
    
    P(word | context) = ...
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
        delta(float): KneserNey parameter.
    """
        
    def __init__(self, storage, delta=0.5):
        self.__storage = storage
        self.__delta = delta
        self.__Ndots = len(self.__storage[2])
        self.__V = len(self.__storage[1])
        self.__rec_start, self.__smoothing = self.precalc()
        self.__smoothing  =  self.__delta * self.__smoothing /self.__V / self.__Ndots
        for word in self.__rec_start.keys():
            self.__rec_start[word] =  (self.__rec_start[word] - self.__delta) / self.__Ndots
            
        self.__counter_ngram = self.count_begin()
        self.__number_ngram = self.number_begin()
        
    
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
    
    def count_begin(self):
        beginsCounter = Counter()
        for gramSize in range(self.__storage.max_n):
            for ngram in self.__storage[gramSize + 1]:
                beginsCounter[ngram[:-1]] += self.__storage(ngram)
        return beginsCounter
    
    def number_begin(self):
        beginsNum = Counter()
        for Size in range(self.__storage.max_n):
            for ngram in self.__storage[Size + 1]:
                beginsNum[ngram[:-1]] += 1
        return beginsNum
    
    
    
    def precalc(self):
        N_plus = Counter()
        NN = []
        for pair in self.__storage[2].keys():
            NN.append(pair[1])
            N_plus[str(pair[1])] += 1
        return N_plus, len(np.unique(np.array(NN)))
        
    def KN(self, word, context):
        if len(context) == 0:
            prob = self.__rec_start[word] + self.__smoothing
        else:
            count = self.__counter_ngram[context]
            if count == 0:
                prob = self(word, context[1:])
            else:
                s_1 = max(self.__storage(context + (word, )) - self.__delta, 0)
                s_2 = self.__delta * self.__number_ngram[context] * self.KN(word, context[1:])
                prob = (s_1 + s_2) / count
        return prob
    
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not isinstance(word, str):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('word must be a string!')
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        
        ### YOUR CODE HERE
        prob = self.KN(word, context)
        ### END YOUR CODE
        
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [113]:
# Initialize estimator
kn_estimator = KneserNeyProbabilityEstimator(storage)

# Estimating perplexity
print('KN estimator perplexity = {}'.format(perplexity(kn_estimator, test_sents)))
print(kn_estimator.prob('To be'.split()))
print(kn_estimator.prob('To be or not to be'.split()))

KN estimator perplexity = 90.00797094386152
3.901657316659065e-06
1.727320227186788e-13


## Определение языка документа

**Постановка задачи:**  
Одна из задач, которая может быть решена при помощи языковых моделей $-$ **определение языка документа**. Реализуйте два классификатора для определения языка документа:
1. Наивный классификатор, который будет учитывать частотности символов и выбирать язык текста по признаку: распределение частот символов "наиболее похоже" на распределение частот символов в выбранном языке.
2. Классификатор на основе языковых моделей. Сами придумайте, как он должен работать.  
_Подсказка_: лучше считать n-грамы не по словам, а по символам.

---

**Как представлены данные:**  
Во всех текстовых файлах на каждой строчке записано отдельное предложение.
1. В папочке _data_ находятся две папочки: _full_ и _plain_. В _full_ находятся тексты в той форме, что они были взяты из сети, в _plain_ находятся те же самые тексты, но с них сначала была снята диакритика, а затем русский и греческий тексты были транслитерованы в английский.
2. В каждой из папочек _full_ и _plain_ находятся папочки _train_ и _test_.
3. В _train_ находятся файлы с текстами с говорящими именами, например, _ru.txt_, _en.txt_.
4. В _test_ находятся файлы _1.txt_, _2.txt_, $\dots$ в которых хранятся тексты, язык которых нужно определить. В этой же папочке находится файл _ans.csv_, в котором вы можете найти правильные ответ и проверить, насколько хорошо сработали Ваши алгоритмы.

---

**Что нужно сделать:**  
Напишите два своих классификатора (которые описаны в постановке задачи) и получите максимально возможное accuracy на test-сете. Разрешается использовать только _train_ для обучения.

---

**В данном задании мы не предоставляем стартового кода!**

В данном задании я решила создать классификатор языка, который бы классифицировал текст, относительно частоты встречающихся в нем kgram. В частве метрики качества я выбрала логарифм правдаподобия.

In [114]:
import os
from collections import Counter
import codecs
import numpy as np
import math
import pandas as pd
import random

In [115]:
def loglikelihood(model,kgrams):
    probs = map(lambda kgram: model[kgram]+1e-4 , kgrams)
    probs= [math.log(i) for i in probs]
    return np.sum(probs)

def normalize(counter):
    norm= float(sum(counter.values()))
    for key in counter:
        counter[key]/=norm
    return counter

При инициализации  LanguageEstimator подается адрес папки, в которой хранятся текстовые документы для обучения. Для каждого из документов наш классификатор строит языковую модель, перечисляет частоты всех встречающихся kgram.
Для предсказания языков классификатору на вход подается адрес папки, в которой содержатся документы, языки которых требуется предсказать, а также список ответов. Классификатор определяет языки всех документов, а затем считает accuracy.

In [116]:
class LanguageEstimator:
    """Class which predicts the language of the given text
    according to the frequency of kgrams (letter kgrams)
    """
        
    def __init__(self, dirname, k = 1):
        language_model = {}
    
        self.__k = k
        for fname in os.listdir(dirname):
            faddr = os.path.join(dirname,fname)
    
            if not fname.endswith('.txt'):
                continue
    
            with codecs.open(faddr,encoding='utf-8') as textfile:
                text = textfile.read()
    
            #k = 1
            kgrams = [text[i:i+k] for i in range(len(text)-k)]
            model = normalize(Counter(kgrams))
            lang_name = fname[:-4]
            language_model[lang_name] = model
        self.__language_model = language_model
        print(self.__language_model.keys())
    
    def __call__(self, testdirname, ans):
        k = self.__k
        true_pred = 0
        name_list = os.listdir(testdirname)
        for fname in name_list:   
            name = fname[:-4]
            faddr = os.path.join(testdirname,fname)
    
            with codecs.open(faddr, encoding='utf-8') as textfile:
                text = textfile.read()
    
            kgrams = [text[i:i+k] for i in range(len(text)-k)]

            est_lang = 'pl'
            est_log = loglikelihood(self.__language_model[est_lang],kgrams)
            for pred_lang in self.__language_model.keys():
                pred_log = loglikelihood(self.__language_model[pred_lang],kgrams)
                if  pred_log > est_log:
                    est_log = pred_log
                    est_lang = pred_lang
            print(fname, est_lang, ans_df[1][int(name)], est_log)
            print(text[:100])
            if (est_lang == ans_df[1][int(name)]):
                true_pred += 1
    
    
            print('\n')
        accuracy = 1.0 * true_pred/len(name_list)
        return(accuracy)
    
    

Для "упрощенного набора" из папки plain я создала языковую модель, классифицирующую тексты на основе частотности однограммов (то есть символов).

In [117]:
ans_df = pd.DataFrame.from_csv("C:\\Users\\Maria\\Desktop\\data\\ans.csv",header=None)

In [118]:
dirname = 'C:\\Users\\Maria\\Desktop\\data\\plain\\train'
language_model_uni = LanguageEstimator(dirname, 1)

dict_keys(['ru', 'fr', 'el', 'sv', 'it', 'nl', 'eo', 'pl', 'no', 'es', 'ca', 'de', 'en', 'fi', 'pt', 'hu'])


Посмотрим, как она работает. При предсказании я печатала первые строки документа, его намер, предсказанный язык и язык, к которому действительно относится документ. Также выводила на печать логарифм максимального правдоподобия.

In [119]:
testdirname = 'C:\\Users\\Maria\\Desktop\\data\\plain\\test'
accuracy = language_model_uni(testdirname, ans_df)

1.txt ca ca -11134.1486095
-Alla on caura aquesta sageta, enterreu-hi el pobre Robin Hood sota el bosc que treu ufana.
-Gaire… 


10.txt ca ca -57426.018903
-Allargueu els ricos al guanet de l'arbre mestre!
Un pasme intrigat es desvetlla en tota cara de l'a


100.txt fi fi -51103.9817789
Kuja johti suurehkolle ruohokentalle, ja talo oli edessamme.
Times-lehtea lukevat harvoin muut kuin 


101.txt fi fi -41496.8953159
Meilla on kumminkin useita johtolankoja kasissamme, ja on hyvin luultavaa, etta joku niista vie totu


102.txt fi fi -475.344704392
Tahtoisitko Bradleyn ohi kulkiessasi pyytaa hanta lahettamaan naulan vakevinta tupakkaansa tanne min


103.txt fi fi -33173.2669178
"En joutanut edes panemaan hattua paahani.
"Kysymys on kai jostakin salametsastyksesta, luulisin", s


104.txt fi fi -5280.62632089
Yhdessa kiiruhdimme alas portaita kadulle.
Mina muistin tuon omituisen varotuksen, joka oli leikattu


105.txt fi fi -58731.2404045
Ja tuon liekin luona nyt tama roisto loikoo odottam

Вау! Accuracy составляет 1. Поразительно, но если посмотреть на табличку сверху, то действительно оказывается, что данный классификатор безошибочно распознавал все документы.

In [120]:
print('Accuracy = {}'.format(accuracy))

Accuracy = 1.0


Теперь попробуем разобраться с full. Для классификации данной выборки я создала модель, работающую на триграмах (3 символах). Посмотрим, как она справится с задачей.

In [121]:
dirname = 'C:\\Users\\Maria\\Desktop\\data\\full\\train'
language_model_tri = LanguageEstimator(dirname, 3)

dict_keys(['ru', 'fr', 'el', 'sv', 'it', 'nl', 'eo', 'pl', 'no', 'es', 'ca', 'de', 'en', 'fi', 'pt', 'hu'])


In [ ]:
ans_df_full = pd.DataFrame.from_csv("C:\\Users\\Maria\\Desktop\\data\\full\\ans.csv",header=None)

In [ ]:
accuracy_full = language_model_tri(testdirname, ans_df_full)

1.txt ca ca -25899.400384
-Alla on caura aquesta sageta, enterreu-hi el pobre Robin Hood sota el bosc que treu ufana.
-Gaire… 


10.txt ca ca -132580.167562
-Allargueu els ricos al guanet de l'arbre mestre!
Un pasme intrigat es desvetlla en tota cara de l'a


100.txt fi fi -123255.401592
Kuja johti suurehkolle ruohokentalle, ja talo oli edessamme.
Times-lehtea lukevat harvoin muut kuin 


101.txt fi fi -99796.37974
Meilla on kumminkin useita johtolankoja kasissamme, ja on hyvin luultavaa, etta joku niista vie totu


102.txt fi fi -1124.89597816
Tahtoisitko Bradleyn ohi kulkiessasi pyytaa hanta lahettamaan naulan vakevinta tupakkaansa tanne min


103.txt fi fi -80203.8613288
"En joutanut edes panemaan hattua paahani.
"Kysymys on kai jostakin salametsastyksesta, luulisin", s


104.txt fi fi -12723.9091753
Yhdessa kiiruhdimme alas portaita kadulle.
Mina muistin tuon omituisen varotuksen, joka oli leikattu


105.txt fi fi -141494.118059
Ja tuon liekin luona nyt tama roisto loikoo odottamas

Да, точность здесь уже оказывается не 100%, наш классификатор допускает ошибки. Однако, все равно процент правильно определенных текстов очень высокий.

In [ ]:
print('Accuracy (for full test) = {}'.format(accuracy_full))